# 特性与描述符

除了属性之外,我们还可以创建特性(property),在不改变类接口的前提下,使用存取方法(即读值方法和设值方法)修改数据属性.这与统一访问原则相符--不管服务是由存储还是计算实现的,一个模块提供的所有服务都应该通过统一的方式使用.


`property`是一个用于类中方法的装饰器,用于将方法属性转换为特性,如果要设定特性的增删改查能力,则可以使用`<property>.setter,<property>.deleter`定义.

```python
class Event(DbRecord): 
    @property
    def venue(self):
    '''The Event attribute'''
        return self.__venue
        
    @venue.setter
    def venue(self,value):
        self.__venue = value
        
    @venue.deleter
    def venue(self,value):
        del self.__venue     
    
```

虽然内置的`property`经常用作装饰器,但它其实是一个类.在Python中,函数和类通常可以互换,因为二者都是可调用的对象,而且没有实例化对象的`new`运算符,所以调用构造方法与调用工厂函数没有区别.此外,只要能返回新的可调用对象,代替被装饰的函数,二者都可以用作装饰器.

`property`构造方法的完整签名如下：

`property(fget=None, fset=None, fdel=None, doc=None)`

所有参数都是可选的,如果没有把函数传给某个参数,那么得到的特性对象就不允许执行相应的操作.

某些情况下,这种经典形式比装饰器句法好.但是在方法众多的类定义体中使用装饰器的话,一眼就能看出哪些是读值方法,哪些是设值方法,而不用按照惯例在方法名的前面加上`get`和`set`.类中的特性能影响实例属性的寻找方式,而一开始这种方式可能会让人觉得意外.

特性都是类属性,但是特性管理的其实是实例属性的存取.如果实例和所属的类有同名数据属性,那么实例属性会覆盖(或称遮盖)类属性--至少通过那个实例读取属性时是这样.

本节的先验知识有:

+ [面向对象惯用法](/语法篇/面向对象惯用法/)
+ [python的数据模型](/语法篇/Python的数据模型/数据模型.html)
+ [装饰器](/语法篇/元编程/装饰器.html)

## 实例属性遮盖类的数据属性

In [1]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [2]:
obj = Class()
vars(obj)

{}

In [3]:
obj.data

'the class data attr'

In [4]:
obj.data = 'bar'
vars(obj)

{'data': 'bar'}

In [5]:
obj.data

'bar'

In [6]:
Class.data

'the class data attr'

## 实例属性不会遮盖类特性

In [7]:
Class.prop

In [8]:
obj.prop

'the prop value'

In [9]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [10]:
obj.__dict__['prop'] = 'foo'

In [11]:
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [12]:
obj.prop

'the prop value'

In [13]:
Class.prop

In [14]:
obj.prop

'the prop value'

## 新添的类特性遮盖现有的实例属性

In [15]:
obj.data

'bar'

In [16]:
Class.data

'the class data attr'

In [17]:
Class.data = property(lambda self: 'the "data" prop value')
obj.data

'the "data" prop value'

In [18]:
del Class.data
obj.data

'bar'

## 特性的文档

控制台中的`help()`函数或IDE等工具需要显示特性的文档时,会从特性的`__doc__`属性中提取信息.

+ 如果使用经典调用句法,为`property`对象设置文档字符串的方法是传入`doc`参数:
    
    `weight = property(get_weight, set_weight, doc='weight in kilograms')`

+ 使用装饰器创建`property`对象时,读值方法(有`@property`装饰器的方法)的文档字符串作为一个整体,变成特性的文档.

## 使用特性获取链接的记录

下图是用到的几个类
![](source/record.PNG)

+ Record
    
    `__init__` 方法与schedule1.py 脚本（见示例19-9）中的一样;为了辅助测试,增加了`__eq__`方法.

+ DbRecord

    `Record`类的子类,添加了`__db` 类属性,用于设置和获取`__db`属性的`set_db` 和`get_db`静态方法,用于从数据库中获取记录的fetch类方法,以及辅助调试和测试的`__repr__`实例方法.

+ Event

    `DbRecord`类的子类,添加了用于获取所链接记录的`venue`和`speakers`属性,以及特殊的`__repr__`方法.

In [19]:
import inspect
DB_NAME = 'schedule2_db' 
CONFERENCE = 'conference.115'
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    def __eq__(self, other): 
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented
        
class MissingDatabaseError(RuntimeError):
    """需要数据库但没有指定数据库时抛出。"""
    pass

class DbRecord(Record):
    __db = None
    @staticmethod 
    def set_db(db):
        DbRecord.__db = db
    @staticmethod
    def get_db():
        return DbRecord.__db
    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else: 
                raise
                    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()
        
class Event(DbRecord): 
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key) 
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'): 
            spkr_serials = self.__dict__['speakers'] 
            fetch = self.__class__.fetch 
            self._speaker_objs = [fetch('speaker.{}'.format(key)) 
                                      for key in spkr_serials] 
        return self._speaker_objs 
    def __repr__(self):
        if hasattr(self, 'name'): 
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()
        
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1] 
        cls_name = record_type.capitalize() 
        cls = globals().get(cls_name, DbRecord) 
        if inspect.isclass(cls) and issubclass(cls, DbRecord): 
            factory = cls 
        else:
            factory = DbRecord 
        for record in rec_list: 
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)

In [20]:
import shelve

In [21]:
db = shelve.open(DB_NAME) 

In [22]:
if CONFERENCE not in db:
    load_db(db)

In [23]:
DbRecord.set_db(db)

In [24]:
event = DbRecord.fetch('event.33950')
event

<Event 'There *Will* Be Bugs'>

In [25]:
event.venue

<DbRecord serial='venue.1449'>

In [26]:
event.venue.name

'Portland 251'

In [27]:
for spkr in event.speakers:
    print('{0.serial}: {0.name}'.format(spkr))

speaker.3471: Anna Martelli Ravenscroft
speaker.5199: Alex Martelli


In [28]:
db.close()

## 使用特性验证属性

目前,我们只介绍了如何使用`@property`装饰器实现只读特性.本节要创建一个可读写的特性

### `LineItem`类第1版：表示订单中商品的类

假设有个销售散装有机食物的电商应用,客户可以按重量订购坚果、干果或杂粮.在这个系统中,每个订单中都有一系列商品,而每个商品都可以使用.

In [29]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [30]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [31]:
raisins.subtotal()

69.5

In [32]:
raisins.weight = -20

In [33]:
raisins.subtotal()

-139.0

这个类没法限制参数.比如作为一个商品订单,它的值可以是负的.

### `LineItem`类第2版：能验证值的特性

In [34]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price
    @property 
    def weight(self):
        return self.__weight 
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value 
        else:
            raise ValueError('value must be > 0')

## 特性工厂函数

我们的`weight` 和`price`有相似的特点,都不能为负.如果一个类有很多这样的特性,那一个一个写特性会很麻烦,因此可以使用特性工厂函数来产生一样特点的特性.


我们将定义一个名为`quantity`的特性工厂函数,取这个名字是因为,在这个应用中要管理的属性表示不能为负数或零的量.下例是`LineItem`类的简洁版,用到了`quantity`特性的两个实例:

+ 一个用于管理`weight`属性，
+ 另一个用于管理`price`属性。


In [35]:
def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [36]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)

In [37]:
nutmeg.weight, nutmeg.price

(8, 13.95)

In [38]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

工厂函数构建的特性利用了特性覆盖实例属性的行为,因此对`self.weight`或`nutmeg.weight`的每个引用都由特性函数处理,只有直接存取`__dict__`属性才能跳过特性的处理逻辑.

在真实的系统中,分散在多个类中的多个字段可能要做同样的验证,此时最好把`quantity`工厂函数放在实用工具模块中,以便重复使用.最终可能要重构那个简单的工厂函数,改成更易扩展的描述符类,然后使用专门的子类执行不同的验证.

## *属性描述符*

描述符是对多个属性运用相同存取逻辑的一种方式,ORM 中的字段类型是往往使用描述符，把数据库记录中字段里的数据与Python对象的属性对应起来.

描述符是实现了特定协议的类,这个协议包括`__get__`、`__set__` 和`__delete__`方法.

`property`类实现了完整的描述符协议.通常可以只实现部分协议.其实我们在真实的代码中见到的大多数描述符只实现了`__get__` 和`__set__`方法,还有很多只实现了其中的一个.描述符是Python的独有特征,不仅在应用层中使用,在语言的基础设施中也有用到.除了特性之外,使用描述符的Python功能还有方法及`classmethod`和`staticmethod`装饰器.理解描述符是精通Python的关键.

### LineItem类第3版：一个简单的描述符

实现了`__get__`、`__set__` 或`__delete__ `方法的类是描述符.描述符的用法是,创建一个实例,作为另一个类的类属性.

我们将定义一个`Quantity`描述符用来代替特性工厂函数,`LineItem`类会用到两个`Quantity`实例:

+ 一个用于管理`weight`属性
+ 另一个用于管理`price`属性

![](source/des.PNG)


从现在开始我会使用下述定义:

+ 描述符类

    实现描述符协议的类.在上图中,是`Quantity`类.

+ 托管类

    把描述符实例声明为类属性的类——上图中的`LineItem`类

+ 描述符实例

    描述符类的各个实例,声明为托管类的类属性.在上图中,各个描述符实例使用箭头和带下划线的名称表示(在UML中下划线表示类属性).与黑色菱形接触的`LineItem`类包含描述符实例.
    
+ 托管实例

    托管类的实例.在这个示例中,`LineItem`实例是托管实例

+ 储存属性

    托管实例中存储自身托管属性的属性.在上图中,`LineItem`实例的`weight` 和`price`属性是储存属性.这种属性与描述符属性不同,描述符属性都是类属性.
    
+ 托管属性

    托管类中由描述符实例处理的公开属性,值存储在储存属性中.也就是说描述符实例和储存属性为托管属性建立了基础.

In [39]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')

各个托管属性的名称与储存属性一样,而且读值方法不需要特殊的逻辑,所以`Quantity`类不需要定义`__get__`方法.

编写`__set__`方法时，要记住`self` 和`instance`参数的意思:

+ self 是描述符实例，
+ instance 是托管实例

管理实例属性的描述符应该把值存储在托管实例中.因此Python才为描述符中的那个方法提供了`instance`参数.

In [40]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [41]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

上面的方式还是不够简洁,我们不得不在申明LineItem时为每个属性指定`Quantity()`的参数--属性的名称.

可问题是赋值语句右手边的表达式先执行,而此时变量还不存在.

`Quantity()`表达式计算的结果是创建描述符实例,而此时`Quantity`类中的代码无法猜出要把描述符绑定给哪个变量(例如`weight`或`price`).

因此上例必须明确指明各个`Quantity`实例的名称.这样不仅麻烦,还很危险--如果程序员直接复制粘贴代码而忘了编辑名称,比如写成`price = Quantity('weight')`，那么程序的行为会大错特错，设置`price`的值时会覆盖`weight`的值.

### LineItem类第4版：自动获取储存属性的名称

为了避免在描述符声明语句中重复输入属性名,我们将为每个`Quantity`实例的`storage_name`属性生成一个独一无二的字符串.下图是更新后的`Quantity `和`LineItem`类的UML类图.

![](source/lineitem4.PNG)

为了生成storage_name,我们以`'_Quantity#'`为前缀,然后在后面拼接一个整数:

`Quantity.__counter`类属性的当前值,每次把一个新的`Quantity`描述符实例依附到类上,都会递增这个值.在前缀中使用井号能避免`storage_name`与用户使用点号创建的属性冲突,因为`nutmeg._Quantity#0`是无效的Python句法.但是,内置的`getattr` 和`setattr`函数可以使用这种"无效的"标识符获取和设置属性,此外也可以直接处理实例属性`__dict__`

In [42]:
class Quantity:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

In [43]:
class LineItem:
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [44]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [45]:
coconuts.weight, coconuts.price

(20, 17.95)

In [46]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

### LineItem类第5版：一种新型描述符

我们虚构的有机食物网店遇到一个问题:

>不知怎么回事儿有个商品的描述信息为空,导致无法下订单.

为了避免出现这个问题,我们要再创建一个描述符`NonBlank`.在设计`NonBlank`的过程中,我们发现它与`Quantity`描述符很像,只是验证逻辑不同.

回想`Quantity`的功能，我们注意到它做了两件不同的事：

+ 管理托管实例中的储存属性
+ 验证用于设置那两个属性的值

由此可知,我们可以重构,并创建两个基类

+ AutoStorage

    自动管理储存属性的描述符类
    
+ Validated

    扩展AutoStorage类的抽象子类,覆盖`__set__` 方法,调用必须由子类实现的`validate`方法

我们重写`Quantity`类,并实现`NonBlank`,让它继承`Validated`类,只编写`validate`方法.类之间的关系见图.

![](source/lineitem5.PNG)

`Validated`、`Quantity`和`NonBlank` 三个类之间的关系体现了模板方法设计模式.具体而言，`Validated.__set__` 方法正是`Gamma`等四人所描述的模板方法的例证--一个模板方法用一些抽象的操作定义一个算法，而子类将重定义这些操作以提供具体的行为.


In [47]:
import abc
class AutoStorage:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

In [48]:
class Validated(abc.ABC, AutoStorage): 
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        pass

In [49]:
class Quantity(Validated): 
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value

In [50]:
class NonBlank(Validated):
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [51]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [52]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [53]:
coconuts.weight, coconuts.price

(20, 17.95)

In [54]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#0', '_Quantity#0', '_Quantity#1']

In [55]:
LineItem.description.storage_name

'_NonBlank#0'

## *覆盖型与非覆盖型描述符*

Python存取属性的方式特别不对等.通过实例读取属性时,通常返回的是实例中定义的属性;但是如果实例中没有指定的属性,那么会获取类属性.而为实例中的属性赋值时,通常会在实例中创建属性,根本不影响类.这种不对等的处理方式对描述符也有影响.其实根据是否定义`__set__`方法,描述符可分为两大类.其中覆盖型又可以分为2小类.

+ 覆盖型

    定义`__set__`,描述符的`__set__`方法使用托管实例中的同名属性覆盖(即插手接管)了要设置的属性,这种类型描述符的典型用途是管理数据属性
    
    + 没有`__get__`方法的覆盖型描述符
    
        通常，覆盖型描述符既会实现`__set__` 方法,也会实现`__get__`方法,不过也可以只实现`__set__` 方法.此时,只有写操作由描述符处理.通过实例读取描述符会返回描述符对象本身,因为没有处理读操作的`__get__` 方法.如果直接通过实例的`__dict__`属性创建同名实例属性,以后再设置那个属性时,仍会由`__set__` 方法插手接管,但是读取那个属性的话,就会直接从实例中返回新赋予的值,而不会返回描述符对象.也就是说实例属性会遮盖描述符,不过只有读操作是如此
    
    
+ 非覆盖型

    没有实现`__set__`方法的描述符是非覆盖型描述符.如果设置了同名的实例属性,描述符会被遮盖,致使描述符无法处理那个实例的那个属性.方法是以非覆盖型描述符实现的

我们通过下面的例子观察这两类描述符的行为差异

In [56]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]
def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))

In [57]:
class Overriding:
    """覆盖型描述符
    也称数据描述符或强制描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [58]:
class OverridingNoGet:
    """没有`__get__`方法的覆盖型描述符"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [59]:
class NonOverriding:
    """也称非数据描述符或遮盖型描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

In [60]:
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

### 覆盖型描述符的行为

上面的例子都是覆盖型描述符

In [61]:
obj = Managed()

In [62]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [63]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [64]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [65]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [66]:
obj.__dict__['over'] = 8

In [67]:
vars(obj)

{'over': 8}

In [68]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


### 没有`__get__`的覆盖型描述符的行为

只有写操作由描述符处理.通过实例读取描述符会返回描述符对象本身

In [69]:
obj.over_no_get

In [70]:
Managed.over_no_get

In [71]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [72]:
obj.over_no_get

In [73]:
obj.__dict__['over_no_get'] = 9

In [74]:
obj.over_no_get

9

In [75]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [76]:
obj.over_no_get

9

### 非覆盖型描述符的行为

如果设置了同名的实例属性,描述符会被遮盖,致使描述符无法处理那个实例的那个属性

In [77]:
obj = Managed()

In [78]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [79]:
obj.non_over = 7

In [80]:
obj.non_over

7

In [81]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [82]:
del obj.non_over

In [83]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


## *在类中覆盖描述符*

依附在类上的描述符无法控制为类属性赋值的操作.其实,这意味着为类属性赋值能覆盖描述符属性.这是一种猴子补丁技术,不过在下例中,我们把描述符替换成了整数,这其实会导致依赖描述符的类不能正确地执行操作.

In [84]:
obj = Managed()

In [85]:
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over

(1, 2, 3)

读类属性的操作可以由依附在托管类上定义有`__get__` 方法的描述符处理,但是写类属性的操作不会由依附在托管类上定义有`__set__`方法的描述符处理.

若想控制设置类属性的操作,要把描述符依附在类的类上,即依附在元类上.默认情况下,对用户定义的类来说,其元类是`type`,而我们不能为`type` 添加属性,但我们可以自定义元类.

## *描述符协议增强[3.6]*

上面的`LineItem`有个缺陷--就是初始化的时候都明确让属性的值绑定在Integer上的name属性上,而无法获知所有者类的属性名.如果使用自定义内部名字,又会难以调试.使用在[PEP487](https://www.python.org/dev/peps/pep-0487/)上提供的可选的`__set_name__()`可以获得这个属性名字,并且可以自定义这部分内容:

In [86]:
class AutoStorage:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.name)
    def __set__(self, instance, value):
        setattr(instance, self.name, value) 
    def __set_name__(self, owner, name):
        cls = self.__class__
        prefix = cls.__name__
        index = name
        self.name = '_{}#{}'.format(prefix, index)
        
class Validated(abc.ABC, AutoStorage): 
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        pass
class Quantity(Validated): 
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    
class NonBlank(Validated):
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [87]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [88]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [89]:
LineItem.description.name

'_NonBlank#description'

## *方法是描述符*

在类中定义的函数属于绑定方法(bound method),因为用户定义的函数都有`__get__`方法,所以依附到类上时,就相当于描述符.函数没有实现`__set__`方法,因此是非覆盖型描述符.

与描述符一样,通过托管类访问时,函数的`__get__`方法会返回自身的引用.但是通过实例访问时,函数的`__get__`方法返回的是绑定方法对象--一种可调用的对象,里面包装着函数,并把托管实例(例如`obj`)绑定给函数的第一个参数(即`self`),这与`functools.partial`函数的行为一致

## *描述符用法建议*

下面根据刚刚论述的描述符特征给出一些实用的结论:

+ 使用特性以保持简单

    内置的property 类创建的其实是覆盖型描述符,`__set__ `方法和`__get__` 方法都实现了,即便不定义设值方法也是如此.特性的`__set__ `方法默认抛出`AttributeError:can't set attribute`,因此创建只读属性最简单的方式是使用特性,这能避免下一条所述的问题.

+ 只读描述符必须有`__set__`方法

    如果使用描述符类实现只读属性,要记住`__get__` 和`__set__` 两个方法必须都定义,否则实例的同名属性会遮盖描述符.只读属性的`__set__`方法只需抛出`AttributeError`异常,并提供合适的错误消息.
    
+ 用于验证的描述符可以只有`__set__`方法

    对仅用于验证的描述符来说,`__set__` 方法应该检查value参数获得的值,如果有效,使用描述符实例的名称为键,直接在实例的`__dict__`属性中设置.这样从实例中读取同名属性的速度很快,因为不用经过`__get__`方法处理.
    
+ 仅有`__get__`方法的描述符可以实现高效缓存

    如果只编写了`__get__`方法,那么创建的是非覆盖型描述符.这种描述符可用于执行某些耗费资源的计算,然后为实例设置同名属性,缓存结果.同名实例属性会遮盖描述符,因此后续访问会直接从实例的`__dict__`属性中获取值,而不会再触发描述符的`__get__`方法.

+ 非特殊的方法可以被实例属性遮盖

    由于函数和方法只实现了`__get__` 方法,它们不会处理同名实例属性的赋值操作.因此，像`my_obj.the_method = 7` 这样简单赋值之后,后续通过该实例访问`the_method`得到的是数字7——但是不影响类或其他实例.然而,特殊方法不受这个问题的影响.解释器只会在类中寻找特殊的方法,也就是说`repr(x)`执行的其实是`x.__class__.__repr__(x)`,因此x的`__repr__` 属性对`repr(x)`方法调用没有影响.出于同样的原因,实例的`__getattr__`属性不会破坏常规的属性访问规则.